In [39]:
import wmfdata as wmf

In [40]:
def read_file(path):
    with open(path, 'r') as f:
        return f.read()

# Analytics Data Lake

Data from the usual data source.

In [55]:
glob_ret = wmf.hive.run(
    """
    select 
        cohort as reg_mo,
        count(*) as new_eds,
        (sum(if(2nd_month_edits >= 1, 1, 0)) / sum(if(1st_month_edits >= 1, 1, 0))) as ret 
    from neilpquinn.new_editors
    group by cohort
    order by cohort asc
    limit 10000
    """
)

glob_ret.head()

,reg_mo,new_eds,ret
0,2001-01,6,0.500000
1,2001-02,5,0.600000
2,2001-03,13,0.538462
3,2001-04,11,0.272727
4,2001-05,13,0.615385


In [56]:
glob_ret.to_csv("data/adl_global_monthly_retention.tsv", sep = "\t", index = False)

In [57]:
proj_ret = wmf.hive.run(
    """
    select 
        wiki,
        cohort as reg_mo,
        count(*) as new_eds,
        (sum(if(2nd_month_edits >= 1, 1, 0)) / sum(if(1st_month_edits >= 1, 1, 0))) as ret
    from neilpquinn.new_editors
    group by wiki, cohort
    order by wiki, cohort asc
    limit 250000
    """
)

In [58]:
proj_ret.head()

,wiki,reg_mo,new_eds,ret
0,abwiki,2004-01,1,0.0
1,abwiki,2004-08,1,0.0
2,abwiki,2004-11,2,0.0
3,abwiki,2004-12,2,0.5
4,abwiki,2005-01,1,0.0


In [65]:
proj_ret.query("wiki == 'enwiki'").tail(20)

,wiki,reg_mo,new_eds,ret
14554,enwiki,2016-07,40900,0.049560
14555,enwiki,2016-08,42999,0.061932
14556,enwiki,2016-09,47762,0.077174
14557,enwiki,2016-10,49119,0.073882
14558,enwiki,2016-11,49645,0.044858
14559,enwiki,2016-12,45125,0.048355
14560,enwiki,2017-01,55376,0.083394
14561,enwiki,2017-02,53840,0.061163
14562,enwiki,2017-03,58095,0.056390
14563,enwiki,2017-04,51073,0.054079


In [59]:
proj_ret.to_csv("data/adl_project_monthly_retention.tsv", sep = "\t", index = False)

In [44]:
adl_en_new = wmf.hive.run(
    read_file("queries/adl_enwiki_new_editors.sql").format(
        start = "2016-12",
        end = "2018-03",
        snapshot = "2018-04"
    )
)

adl_en_new.head()

,user,reg_day,mo_1_revs,mo_2_revs,mo_1_reverts,mo_2_reverts
0,!!Ethereal!!,2018-02-06,2,0,0,0
1,!!Hûm Såfår!!,2017-02-23,4,0,1,0
2,!.Jimiey.!,2016-12-17,2,0,2,0
3,!0Clarke,2018-01-06,3,0,1,0
4,!0MinutesADay,2017-11-17,1,0,0,0


In [76]:
adl_en_new = adl_en_new.rename(columns = {"user": "name"})

In [77]:
adl_en_new.head()

,name,reg_day,mo_1_revs,mo_2_revs,mo_1_reverts,mo_2_reverts
0,!!Ethereal!!,2018-02-06,2,0,0,0
1,!!Hûm Såfår!!,2017-02-23,4,0,1,0
2,!.Jimiey.!,2016-12-17,2,0,2,0
3,!0Clarke,2018-01-06,3,0,1,0
4,!0MinutesADay,2017-11-17,1,0,0,0


In [78]:
adl_en_new.to_csv("data/adl_enwiki_new_editors.tsv", sep = "\t", index = False)

# MariaDB application tables
Raw data for comparison

In [ ]:
mdb_en_new_revs = wmf.mariadb.run(
    read_file('queries/mdb_enwiki_new_editor_revisions.sql')
)

mdb_en_new_revs.head()

In [70]:
mdb_en_new_revs.head()

,name,rev_id,reg_dt,rev_dt,mob,ve
0,HappyPaige627,752388509,2016-12-01 00:00:15,2016-12-01 00:01:48,0,0
1,Toppu Kaeru,752389022,2016-12-01 00:01:09,2016-12-01 00:04:02,0,0
2,GAdams,752389051,2016-12-01 00:03:08,2016-12-01 00:04:09,1,0
3,Albitr,752389167,2016-12-01 00:00:34,2016-12-01 00:04:44,0,1
4,Ltclaws,752390193,2016-12-01 00:02:26,2016-12-01 00:09:38,0,1


In [54]:
mdb_en_new_revs.to_csv("data/mdb_enwiki_new_editor_revisions.tsv", index = False, sep = "\t")